In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,47446
2,Huelva,Confirmados PDIA 14 días,817
3,Huelva,Tasa PDIA 14 días,"159,20650076972544"
4,Huelva,Confirmados PDIA 7 días,457
5,Huelva,Tasa PDIA 7 dias,"89,05430948808387"
6,Huelva,Total Confirmados,47649
7,Huelva,Curados,45746
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  47446.0


/tmp/ipykernel_77658/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13143.0


/tmp/ipykernel_77658/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_77658/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_77658/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_77658/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 899 personas en los últimos 7 días 

Un positivo PCR cada 457 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,47446.0,457.0,817.0,513170.0,89.054309,159.206501,118.0
Huelva-Costa,28045.0,261.0,488.0,289548.0,90.140495,168.538550,59.0
Condado-Campiña,14796.0,157.0,276.0,156231.0,100.492220,176.661482,46.0
Huelva (capital),13143.0,160.0,315.0,143837.0,111.237025,218.997893,30.0
Sierra de Huelva-Andévalo Central,4177.0,39.0,52.0,67391.0,57.871229,77.161639,13.0
Cartaya,2251.0,38.0,77.0,20083.0,189.214759,383.408853,10.0
Moguer,2116.0,37.0,57.0,21867.0,169.204738,260.666758,8.0
Aljaraque,1689.0,26.0,38.0,21474.0,121.076651,176.958182,8.0
Valverde del Camino,880.0,15.0,18.0,12750.0,117.647059,141.176471,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),15.0,8.0,9.0,243.0,3292.181070,3703.703704,1.0
Paterna del Campo,239.0,19.0,24.0,3457.0,549.609488,694.243564,7.0
Palos de la Frontera,1177.0,29.0,60.0,11742.0,246.976665,510.986203,5.0
Cartaya,2251.0,38.0,77.0,20083.0,189.214759,383.408853,10.0
Alosno,248.0,12.0,14.0,3933.0,305.110603,355.962370,0.0
Campofrío,62.0,2.0,2.0,713.0,280.504909,280.504909,1.0
Palma del Condado (La),1474.0,15.0,30.0,10801.0,138.876030,277.752060,6.0
Moguer,2116.0,37.0,57.0,21867.0,169.204738,260.666758,8.0
Cañaveral de León,34.0,1.0,1.0,396.0,252.525253,252.525253,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,398.0,1.0,7.0,3761.0,26.588673,186.120713,0.0,0.142857
Bonares,474.0,1.0,6.0,6060.0,16.501650,99.009901,1.0,0.166667
Isla Cristina,3196.0,2.0,10.0,21393.0,9.348852,46.744262,0.0,0.200000
Gibraleón,968.0,4.0,10.0,12737.0,31.404569,78.511423,1.0,0.400000
Trigueros,555.0,4.0,9.0,7862.0,50.877639,114.474688,3.0,0.444444
Palos de la Frontera,1177.0,29.0,60.0,11742.0,246.976665,510.986203,5.0,0.483333
Cartaya,2251.0,38.0,77.0,20083.0,189.214759,383.408853,10.0,0.493506
Villanueva de los Castillejos,209.0,1.0,2.0,2825.0,35.398230,70.796460,0.0,0.500000
Almonte,2371.0,9.0,18.0,24507.0,36.724201,73.448402,0.0,0.500000
